In [16]:
import bhnerf
from astropy import units
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
import ruamel.yaml as yaml

data_path = Path('../data/synthetic_lightcurves')

# Generate 3D emissivities
---
Generate different emissivity structures for recovery:
 - Single hotspot
 - Three Hotspots
 - Hotpot + tube

### Single Gaussian hotspot

In [24]:
fov_M = 40.0
emission_scale = 3.0   
hs_fwhm = 3
hs_std = hs_fwhm / 2.355
orbit_radius = 11.0
rot_angle = np.deg2rad(120) + np.pi/2
emission = emission_scale * bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=rot_angle,
    orbit_radius=orbit_radius,
    std=hs_std,
    r_isco=6.0,
    fov=(fov_M, 'GM/c^2'))
emission.name = 'Single Gaussian'
emission.attrs = {
    'fov_M': fov_M,
    'hs_fwhm': hs_fwhm,
    'orbit_radius': orbit_radius,
    'rot_angle': rot_angle,
    'emission_scale': emission_scale}

# Save ground truth emission 
directory = data_path.joinpath('single_gaussian')
directory.mkdir(parents=True, exist_ok=True)
emission.to_netcdf(directory.joinpath('emission_3d.nc'))

In [20]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

### Three Gaussian hotspots

In [3]:
fov_M = 40.0
emission_scale = 1
hs_fwhms = np.array([3.4, 4.2, 4.6])
hs_stds = hs_fwhms / 2.355
orbit_radii = [6.5, 11.0, 13.0]
rot_angles = np.deg2rad(np.array([9.5, 72.3, 221.8]))

emission = 0
for hs_std, orbit_radius, rot_angle in zip(hs_stds, orbit_radii, rot_angles):
    emission += emission_scale * bhnerf.emission.generate_hotspot_xr(
        resolution=(64, 64, 64), 
        rot_axis=[0.0, 0.0, 1.0], 
        rot_angle=rot_angle,
        orbit_radius=orbit_radius,
        std=hs_std,
        r_isco=6.0,
        fov=(fov_M, 'GM/c^2'))
emission.name = 'Three Gaussians'
emission.attrs = {
    'fov_M': fov_M,
    'hs_fwhm': hs_fwhms,
    'orbit_radius': orbit_radii,
    'rot_angle': rot_angles,
    'emission_scale': emission_scale}

# Save ground truth emission 
directory = data_path.joinpath('3_gaussians')
directory.mkdir(parents=True, exist_ok=True)
emission.to_netcdf(directory.joinpath('emission_3d.nc'))

In [6]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

### Flux Tube

In [91]:
fov_M = 40.0
hs_std = 1.2
tube_std = 1.
tube_radius = 10.0
phi_start = np.deg2rad(190)
phi_end = np.deg2rad(270)
emission = 4

emission = emission * bhnerf.emission.generate_tube_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    phi_start=phi_start,
    phi_end=phi_end,
    orbit_radius=tube_radius,
    std=tube_std,
    r_isco=6.0,
    fov=(fov_M, 'GM/c^2'))
emission.name = 'Flux Tube'
emission.attrs = {
    'fov_M': fov_M,
    'hs_std': hs_std,
    'tube_std':tube_std,
    'tube_radius':tube_radius,
    'tube_scale': tube_scale}


# Save ground truth emission 
directory = data_path.joinpath('flux_tube')
directory.mkdir(parents=True, exist_ok=True)
emission.to_netcdf(directory.joinpath('emission_3d.nc'))

In [80]:
%matplotlib widget
bhnerf.visualization.ipyvolume_3d(emission, fov_M)

# Generate lightcurves 
---
Polarized image-plane/lightcurves corresponding to ALMA scan times on April 11, 2017.

In [92]:
simulation_params = {
    'name': 'sim1',
    'desc': 'Image fluxes without background accretion',
    'flare_path': '../data/synthetic_lightcurves/flux_tube/emission_3d.nc',
    'model': {
        # Domain dimensions and black-hole params
        'spin': 0.0,
        'inclination': 12.0,
        'fov_M': 40.0, 
        # Magnetic field / polarization parameters
        'Q_frac': 0.85,      # Fraction of linear polarization
        'b_consts': {
            'arad': 0,       # Radial component (in lab-frame) 
            'avert': 1,      # Vertical component (in lab-frame)
            'ator': 0,       # Toroidal component (in lab-frame)
        },
        # Keplerian velocity field
        'Omega_dir': 'cw',
        # Image plane 
        'num_alpha': 64,
        'num_beta': 64,
        't_start_obs': 9.25 
    }}
locals().update(simulation_params['model'])

In [93]:
""" Flare emissivity """
sim_name = simulation_params['name']
flare_path = Path(simulation_params['flare_path'])
emission_flare = xr.load_dataarray(flare_path)

# Compute geodesics
geos = bhnerf.kgeo.image_plane_geos(
    spin, np.deg2rad(inclination), 
    num_alpha=num_alpha, num_beta=num_beta, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2])
t_injection = -float(geos.r_o + fov_M/4)


# Keplerian velocity field
rot_sign = {'cw': -1, 'ccw': 1}
Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
g = bhnerf.kgeo.doppler_factor(geos, umu)

# Magnitude normalized magnetic field in fluid-frame
b = bhnerf.kgeo.magnetic_field_fluid_frame(geos, umu, **b_consts)
z_width = 4                                      # maximum disk width [M]
rmax = fov_M / 2                                 # maximum recovery radius
rmin = float(bhnerf.constants.isco_pro(spin))    # minimum recovery radius
domain = np.bitwise_and(np.bitwise_and(np.abs(geos.z) < z_width, geos.r > rmin), geos.r < rmax)
b_mean = np.sqrt(np.sum(b[domain]**2, axis=-1)).mean()
b /= b_mean

# Polarized emission factors (including parallel transport)
J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)

# ALMA scans on Apr11, 2017.
t_frames = np.array([ 9.34056333,  9.35067   ,  9.36077667,  9.37088333,  9.38099   ,
                      9.39109667,  9.40120333,  9.41131   ,  9.42169667,  9.43405   ,
                      9.44415667,  9.45426333,  9.46437   ,  9.47447667,  9.48458333,
                      9.49546167,  9.56607333,  9.57618   ,  9.58628667,  9.59639333,
                      9.6065    ,  9.61660667,  9.62671333,  9.63682   ,  9.64889333,
                      9.65956   ,  9.66966667,  9.67977333,  9.68988   ,  9.69998667,
                      9.71191667,  9.91607333,  9.92618   ,  9.93881333,  9.94892   ,
                      9.95902667,  9.96913333,  9.97924   ,  9.98962667, 10.00071667,
                     10.01082333, 10.02093   , 10.03103667, 10.04114333, 10.05139   ,
                     10.06262   , 10.13274   , 10.14284667, 10.15295333, 10.16306   ,
                     10.17316667, 10.18327333, 10.19338   , 10.20348667, 10.21359333,
                     10.2237    , 10.23507   , 10.24644   , 10.28883833, 10.30975667,
                     10.31986333, 10.32997   , 10.34007667, 10.35018333, 10.36029   ,
                     10.37039667, 10.381415  , 10.39187333, 10.40198   , 10.41405333,
                     10.42472   , 10.43482667, 10.44493333, 10.45504   , 10.73603833,
                     10.74663667, 10.75674333, 10.76685   , 10.77695667, 10.78706333,
                     10.79808167, 10.80854   , 10.81977   , 10.83632167, 10.90067   ,
                     10.91077667, 10.92088333, 10.93211333, 10.94236   , 10.95246667,
                     10.96257333, 10.97345167, 10.98405   , 10.99415667, 11.00426333,
                     11.01437   , 11.02447667, 11.03584667]) * units.hr
image_plane_flare = bhnerf.emission.image_plane_dynamics(
    emission_flare, geos, Omega, t_frames, t_injection, J, t_start_obs=t_start_obs*units.hr
)

""" Background disk emissivity """
image_plane_disk = 0.0

""" Image-plane fluxes """
image_plane = image_plane_disk + image_plane_flare
lightcurves = image_plane.sum(axis=(-1,-2))

# Save image_plane, light curves and simulation parameters
image_plane_xr = xr.DataArray(image_plane, {'t': t_frames, 'stokes': ['I', 'Q', 'U'], 'beta': geos.beta, 'alpha': geos.alpha}, ['t', 'stokes', 'beta', 'alpha'], 'Image Plane')
lightcurves_xr =  xr.DataArray(lightcurves, {'t': t_frames, 'stokes': ['I', 'Q', 'U']}, ['t', 'stokes'], 'lightcurves')
image_plane_xr.to_netcdf(flare_path.parent.joinpath(sim_name+'_image_plane.nc'))
lightcurves_xr.to_pandas().to_csv(flare_path.parent.joinpath(sim_name+'_lightcurve.csv'))
with open(flare_path.parent.joinpath('{}_params.yml'.format(sim_name)), 'w') as file:
    yaml.dump(simulation_params, file, default_flow_style=False)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/scratch/imaging/projects/bh_nerf/bhNeRF/bhnerf/kgeo.py:482: RuntimeWarning: invalid value encountered in sqrt
  cot_th_b = np.sqrt(1 - sin_th_b**2) / sin_th_b
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [94]:
%matplotlib widget
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(
    image_plane_xr.transpose('stokes',...), axes, output=flare_path.parent.joinpath(sim_name + '_image_plane.gif'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
%matplotlib widget
axes = bhnerf.visualization.plot_stokes_lc(lightcurves, ['I', 'Q', 'U'], t_frames, add_mean=True)
plt.savefig(flare_path.parent.joinpath(sim_name + '_lightcurve.pdf'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …